In [3]:
import json
import random
from datasets import load_dataset

import warnings
warnings.filterwarnings("ignore")

DATASET_ID = "sahil2801/CodeAlpaca-20k"
SPLIT = "train"
OUTPUT_PATH = "pretrain_code_alpaca20k.jsonl"

MAX_SAMPLES = 20000
MAX_CHARS = 512
CHUNK_SIZE = 2000

BOS = "<|im_start|>"
EOS = "<|im_end|>"

# ================================
# BUILD PROMPT
# ================================
def build_prompt(sample):
    instr = sample.get("instruction", "").strip()
    inp = sample.get("input", "").strip()
    out = sample.get("output", "").strip()

    # merge instruction + input into user message
    user_msg = instr
    if inp:
        user_msg += "\n" + inp

    # Format:
    # <|im_start|>user
    # {user_msg}
    # <|im_end|>
    #
    # <|im_start|>assistant
    # {answer}
    # <|im_end|>

    return (
        f"{BOS}user\n{user_msg}{EOS}\n"
        f"{BOS}assistant\n{out}{EOS}"
    )

# ================================
# CHUNK WRITER
# ================================
def write_chunk(buffer, file_path):
    random.shuffle(buffer)
    with open(file_path, "a", encoding="utf-8") as f:
        for row in buffer:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

# ================================
# MAIN
# ================================
def main():
    print(f"🚀 Loading dataset: {DATASET_ID}")
    dataset = load_dataset(DATASET_ID, split=SPLIT, streaming=False)

    buffer = []
    written = 0
    chunk_id = 1
    total_chars = 0

    for sample in dataset:
        if written >= MAX_SAMPLES:
            break

        prompt = build_prompt(sample)

        # length filtering
        if len(prompt) >= MAX_CHARS:
            continue

        total_chars += len(prompt)
        buffer.append({"text": prompt})
        written += 1

        if len(buffer) >= CHUNK_SIZE:
            print(f"📝 Writing chunk {chunk_id}… total {written} samples")
            write_chunk(buffer, OUTPUT_PATH)
            buffer = []
            chunk_id += 1

    # final write
    if buffer:
        write_chunk(buffer, OUTPUT_PATH)

    # ================================
    # SUMMARY
    # ================================
    print("\n🎉 DONE")
    print(f"📦 Total samples written: {written}")
    print(f"🔣 Total characters: {total_chars:,}")
    est_tokens = total_chars / 4
    print(f"🔢 Estimated tokens: {est_tokens:,.0f}")
    print(f"🔢 Tokens (millions): {est_tokens / 1e6:.3f}M")
    print(f"📄 Saved to: {OUTPUT_PATH}")


if __name__ == "__main__":
    main()

🚀 Loading dataset: sahil2801/CodeAlpaca-20k


code_alpaca_20k.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/20022 [00:00<?, ? examples/s]

📝 Writing chunk 1… total 2000 samples
📝 Writing chunk 2… total 4000 samples
📝 Writing chunk 3… total 6000 samples
📝 Writing chunk 4… total 8000 samples
📝 Writing chunk 5… total 10000 samples
📝 Writing chunk 6… total 12000 samples
📝 Writing chunk 7… total 14000 samples
📝 Writing chunk 8… total 16000 samples

🎉 DONE
📦 Total samples written: 16746
🔣 Total characters: 4,720,438
🔢 Estimated tokens: 1,180,110
🔢 Tokens (millions): 1.180M
📄 Saved to: pretrain_code_alpaca20k.jsonl
